# Model of Spiking Neurons
## Getting Started

The code to generate spiking data from simulations of leaky integrate and fire neurons is can be found in `src/quadratic_integrate_and_fire.py`.
It uses the Brian2 simulator and creates a hdf5 file containing spike times (and meta data) for every set of parameters.

For example,
```
>>> python ./src/quadratic_integrate_and_fire.py -o ./dat/simulations/lif/test.hdf5 -d 300 -equil 60 -r 80
```
will run a simulation that lasts 300 seconds.

Here is a list of the model parameters, that can be specified at run time:
```
>>> python ./src/quadratic_integrate_and_fire.py -h 

  -h, --help            show this help message and exit
  -o FILE               output path
  -jA 45.0              AMPA current strength, in mV
  -jG 50.0              GABA current strength, in mV
  -jM 15.0              Minis (noise amplitude), in mV
  -r 80.0               Poisson rate (minis), all neurons, in Hz
  -tD 20.0              Characteristic recovery time, in seconds
  -s 42                 RNG seed
  -k 5                  Number of bridging axons
  -d 1200               Recording duration, in seconds
  -equil 120, --equilibrate 120
                        Equilibration duration, in seconds
  -stim off             if/how to stimulate: 'off', 'poisson'
  -stim_rate 20         additional rate in stimulated mods, in Hz
  -mod 0                modules to stimulate, e.g. `0`, or `02` for multiple
  --bridge_weight 1.0   synaptic weight of bridge neurons [0, 1]
  --inhibition_fraction 0.2
                        fraction of neurons that should be inhibitory
  --record-state-dt 25.0
                        time step for recording state variables, in ms.
                        use 0.5 to get smooth resource-rate cycles
```

After creating running a simulation, we can use the helper functions in `ana/plot_helper.py` to visualize the results.

In [ ]:
# The autoreload extension allows you to tweak the code in the imported modules
# and rerun cells to reflect the changes.
%load_ext autoreload
%autoreload 2
# this allows us to output the dictionary structure
%load_ext ipy_dict_hierarchy
# matplotlib settings to look decent in the notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# the helpers are in the `/ana/` folder.
import sys
sys.path.append("./../")
from ana import ana_helper as ah
from ana import plot_helper as ph
from ana import paper_plots as pp

# setup logging for notebook
import logging
logging.basicConfig(
    format="%(asctime)s | %(levelname)-8s | %(name)-12s | %(message)s",
    datefmt="%y-%m-%d %H:%M",
    level=logging.INFO,
)
log = logging.getLogger("notebook")

In [ ]:
# lets load and take a look at the file we have created
h5f = ah.prepare_file("./../dat/simulations/lif/test.hdf5")
h5f

The `h5f` above is just a nested dictionary that corresponds to a loaded hdf5 file. We use those a lot throught the analysis and plotting routines. Whenever an analysis is run, it can use details from known places and add the results directly to the `/ana/` dictionary (without writing to disk).

In [ ]:
# silence the detailed outputs
ph.log.setLevel("ERROR")
# and produce overview panels for the loaded file
ph.overview_dynamic(h5f);
ph.overview_topology(h5f);
# Note, when using an interactive matplotlib backend, you can zoom and pan the figures

## Generating Data
We ran two sets of simulations: stimulating the whole system, or only two modules.
For example, the `python ./run/create_pars_for_global_stimulation.py` produces `run/parameters.tsv` with the parameter combinations we used, each line can be called from terminal.
By default, each line corresponds to one realization and produces one hdf5 file (like above) in `dat/simulations/lif/raw`.

Now you would call each line in a bash for-loop, or rather, have a look at `run/submit_to_cluster.sh`
For our example of 2k realizations and an SGE: `qsub -t 1-2000 ./run/submit_to_cluster.sh`

Note on compute time:
We used 50 realizations per coordinate and topology. Each of those (2k in total) runs approximately in real-time, and we simulate 30 minute recordings.

### Data for rate-resource cycles
To get the rate-resource cylces smooth, we have to record the resource variable during the simulations at high time resolution (which needs a lot of disk space, ~4GB for 30 min).
This can be enabled by passing `--record-state-dt 0.5` to `src/quadratic_integrate_and_fire.py`.
If you just want to reproduce the figures from the manuscript, you can quickly run a few simulations:
```
python ./src/quadratic_integrate_and_fire.py -o ./dat/simulations/lif/raw/highres_stim=off_k=-1_jA=45.0_jG=50.0_jM=15.0_tD=20.0_rate=80.0_rep=001.hdf5 -k -1 -d 1800 -equil 300 -s 6102 --record-state-dt 0.5 -r 80
python ./src/quadratic_integrate_and_fire.py -o ./dat/simulations/lif/raw/highres_stim=off_k=-1_jA=45.0_jG=50.0_jM=15.0_tD=20.0_rate=90.0_rep=001.hdf5 -k -1 -d 1800 -equil 300 -s 6102 --record-state-dt 0.5 -r 90
python ./src/quadratic_integrate_and_fire.py -o ./dat/simulations/lif/raw/highres_stim=off_k=1_jA=45.0_jG=50.0_jM=15.0_tD=20.0_rate=80.0_rep=001.hdf5 -k 1 -d 1800 -equil 300 -s 6002 --record-state-dt 0.5 -r 80
python ./src/quadratic_integrate_and_fire.py -o ./dat/simulations/lif/raw/highres_stim=off_k=1_jA=45.0_jG=50.0_jM=15.0_tD=20.0_rate=90.0_rep=001.hdf5 -k 1 -d 1800 -equil 300 -s 6002 --record-state-dt 0.5 -r 90
python ./src/quadratic_integrate_and_fire.py -o ./dat/simulations/lif/raw/highres_stim=off_k=5_jA=45.0_jG=50.0_jM=15.0_tD=20.0_rate=80.0_rep=001.hdf5 -k 5 -d 1800 -equil 300 -s 6102 --record-state-dt 0.5 -r 80
python ./src/quadratic_integrate_and_fire.py -o ./dat/simulations/lif/raw/highres_stim=off_k=5_jA=45.0_jG=50.0_jM=15.0_tD=20.0_rate=90.0_rep=001.hdf5 -k 5 -d 1800 -equil 300 -s 6102 --record-state-dt 0.5 -r 90
python ./src/quadratic_integrate_and_fire.py -o ./dat/simulations/lif/raw/highres_stim=off_k=10_jA=45.0_jG=50.0_jM=15.0_tD=20.0_rate=80.0_rep=001.hdf5 -k 10 -d 1800 -equil 300 -s 6202 --record-state-dt 0.5 -r 80
python ./src/quadratic_integrate_and_fire.py -o ./dat/simulations/lif/raw/highres_stim=off_k=10_jA=45.0_jG=50.0_jM=15.0_tD=20.0_rate=90.0_rep=001.hdf5 -k 10 -d 1800 -equil 300 -s 6202 --record-state-dt 0.5 -r 90
```

## Analysing
### Partial stimulation
The simulations with two targeted modules are analysed by the same script as the experiments
`ana/process_conditions.py`, it produces a pre-stim comparison.

The script has the (input and output) filenames hardcoded. It globs the input files that match the right file pattern, depending on `-t sim_partial`, `-t sim_partial_no_inhib` or `-t sim` (global stimulation) and creates and output file matching the `-t` argument.
For instance,
```
python ./ana/process_conditions.py -t sim_partial -i ./dat/simulations/lif/raw/ -o ./dat/simulations/lif/processed/
```
Scans all files in `/dat/simulations/lif/raw/` and outputs `./dat/simulations/lif/processed/k=5_partial.hdf5`.

### Global stimulation
For the global stimulation, we mainly varied the input rate (`-r`) which ended up on the x-axis of most plots.

In order to combine the many runs in the `dat/simulations/lif/raw/` folder, we use the script in `ana/ndim_merge.py`. It takes a wildcarded file path, runs a lot of analysis, and merges the results into a high-dimensional (x-array like) hdf5 file. As this can take a while, it uses dask and can delegate to a cluster.
```
python ./ana/ndim_merge.py -i './dat/simulations/lif/raw/stim=off*jM=15*tD=20*.hdf5' -o ./dat/simulations/lif/processed/ndim.hdf5 -c num_cores
```

Both of the above scripts are wrappers (handling mostly the IO) around the lower-level analysis routines that can be found in `ana/ana_helper.py`. To reproduce where each observable in the processed dataframes comes from, I recommend to skim the scripts above and then jump to the corresponding part in the `ana_helper`.

## Plots from the paper
### Partial Stimulation

In [ ]:
import numpy as np
from src import topology

topology.log.setLevel("ERROR")
log.setLevel("INFO")

def _ev(x, pdist):
    """
    expectation value via x, p(x)
    """
    ev = 0
    for idx, p in enumerate(pdist):
        ev += x[idx] * pdist[idx]
    return ev


# we want tp find par_alpha so that the expectation value of the in-degree is ~ 30
# iterate until we find a value thats close enough by threshold
def find_by_iteration(generator, target, init=0.0125, iterstep=0.0025, threshold=0.1):
    alpha = init
    prev_was_greater = None
    while True:
        series = generator(alpha)
        # calculate the probability distribution from the series
        pdist = np.bincount(series, minlength=100) / len(series)
        # calculate the expectation value
        ev = _ev(np.arange(len(pdist)), pdist)

        log.debug(f"{alpha=:.4f}, {ev=:.3f}")

        if np.abs(ev - target) < threshold:
            break
        else:
            if prev_was_greater is None:
                prev_was_greater = ev > target
            else:
                if prev_was_greater != (ev > target):
                    iterstep /= 2
                    prev_was_greater = ev > target
                    log.debug(f"new {iterstep=:.3e}")
            # increase or decrease alpha if above or below
            if ev > target:
                alpha -= iterstep
            else:
                alpha += iterstep
    log.info(f"final {alpha=:.4f}, {ev=:.3f}")
    return alpha


def generate_series_of_k_from_topo(alpha, num_connections, num_reps=10):
    """
    generate a series of in-degrees from a topology specified via num_connections.
    -1 for merged
    """
    series = []
    for _ in range(num_reps):
        if num_connections == -1:
            tp = topology.MergedTopology(par_alpha=alpha)
        else:
            tp = topology.ModularTopology(par_k_inter=num_connections, par_alpha=alpha)
        series.extend(tp.k_in)
    return np.array(series)


alphas3 = dict()
for k in [-1, 1, 3, 5, 10, 20]:
    log.info(f"finding alpha for {k=}")
    alpha = find_by_iteration(
        generator=lambda alpha: generate_series_of_k_from_topo(alpha, k),
        target=30,
        threshold=0.1,
    )
    alphas3[k] = alpha

In [ ]:
for k, alpha in alphas.items():
    log.info(f"{k=}, {round(alpha, 5)}")

for k, alpha in alphas2.items():
    log.info(f"{k=}, {round(alpha, 5)}")

for k, alpha in alphas3.items():
    log.info(f"{k=}, {round(alpha, 5)}")

log.info("---")
for k, alpha in alphas3.items():
    log.info(f"{k=}, {round(np.mean([alphas[k], alphas2[k], alphas3[k]]), 5)}")


In [ ]:
pp.log.setLevel("ERROR")
print(pp.fig_3.__doc__)

for k in ["1", "5", "10", "20", "-1"]:
    pp.fig_3(
        pd_path = f"{pp.p_sim}/lif/processed/k={k}_partial.hdf5",
        out_suffix=f"k={k}_partial"
        # raw_paths = [
        #         f"{pp.p_sim}/lif/raw/stim=02_k=5_jA=45.0_jG=50.0_jM=15.0_tD=20.0_rate=80.0_stimrate=0.0_rep=000.hdf5",
        #         f"{pp.p_sim}/lif/raw/stim=02_k=5_jA=45.0_jG=50.0_jM=15.0_tD=20.0_rate=80.0_stimrate=20.0_rep=000.hdf5",
        #     ]
    )

In [ ]:
# to reproduce the figure 3 for the case with blocked inhibition,
# we need to tweak the file paths a bit:
pp.fig_3(
    # processed data frame
    pd_path=f"{pp.p_sim}/lif/processed/k=5_partial_no_inhib.hdf5",
    # raw data for rasters, note jG=0
    raw_paths=[
        f"{pp.p_sim}/lif/raw/stim=02_k=5_jA=45.0_jG=0.0_jM=15.0_tD=20.0_rate=80.0_stimrate=0.0_rep=001.hdf5",
        f"{pp.p_sim}/lif/raw/stim=02_k=5_jA=45.0_jG=0.0_jM=15.0_tD=20.0_rate=80.0_stimrate=20.0_rep=001.hdf5",
    ],

)


In [ ]:
pp.fig_3(
    # processed data frame
    pd_path=f"{pp.p_sim}/lif/processed/k=-1_partial.hdf5",
    # raw data for rasters, note jG=0
    raw_paths=[
        f"{pp.p_sim}/lif/raw/stim=02_k=5_jA=45.0_jG=0.0_jM=15.0_tD=20.0_rate=80.0_stimrate=0.0_rep=001.hdf5",
        f"{pp.p_sim}/lif/raw/stim=02_k=5_jA=45.0_jG=0.0_jM=15.0_tD=20.0_rate=80.0_stimrate=20.0_rep=001.hdf5",
    ],

)

### Stimulation in all modules

In [ ]:
print(pp.fig_4.__doc__)
pp.show_legend = False
pp.fig_4()

In [ ]:
print(pp.fig_4_snapshots.__doc__)
pp.fig_4_snapshots(
    skip_rasters=False,
    # plotting resource cycles takes very long.
    skip_cycles=True,
)

## Inhomogeneous degree distribtions

In [ ]:
# for k in [1, 5, 10, -1]:
#     pp.sim_degrees_sampled(k_inter=k, num_reps=10)

for k in [1]:
    ax = pp.sim_degrees_sampled(k_inter=k, num_reps=10)

In [ ]:
ax.legend()
ax.set_xlim(0,100)